In [1]:
import re
import subprocess
import StringIO
import re

import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp_new import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 

f = open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/one_pdb_chain_features.m', 'r+')
text = f.read()

pdbname = '2TIR'
chain = 'A'

text = re.sub('cluster_PDBname', pdbname, text)
text = re.sub('chain_name', chain, text)
f.seek(0)
f.close()

f2= open('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/one_pdb_chain_features_new.m','w+')
f2.write(text)
f2.close()
#run matlab code for one pdb
subprocess.check_output(["matlab","-nosplash","nodesktop","-wait","-r","run('/home/tongwade780/pdb_website/onebyone_cluster_code/all_function_code/one_pdb_chain_features_new.m')","quit;"])



'Unrecognized MATLAB option "wait".\nWarning: No display specified.  You will not be able to display graphics on the screen.\nWarning: No window system found.  Java option \'MWT\' ignored.\n\n                            < M A T L A B (R) >\n                  Copyright 1984-2013 The MathWorks, Inc.\n                    R2013a (8.1.0.604) 64-bit (glnxa64)\n                             February 15, 2013\n\n \nTo get started, type one of these: helpwin, helpdesk, or demo.\nFor product information, visit www.mathworks.com.\n \nHBPLUS Hydrogen Bond Calculator v 3.2            Jan 18 14:36:51 CST 2018\n(c) I McDonald, D Naylor, D Jones and J Thornton 1993 All Rights Reserved.\n\nConfigured for 100000 atoms and 24000 residues.\nBUG: Residue HOH not recognised in add_donacc.\n\nCriteria\n\nMinimum Angles; DHA 120.00, HAAA  0.00, DAAA  0.00\nMaximum Distances; D-A 3.5, H-A 2.5, S-S 3.0\nMaximum angles at aromatic acceptors DAAX 20.00, HAAAX 20.00\nMinimum covalent separation 3 Covalent bonds\n\n

In [2]:
problem_pdb = []
pdbname= pdbname[0:4].lower()
pdb_file = '{0}_{1}.pdb'.format(pdbname,chain)
pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/one_chain_pdb/'
pdbPATH = os.path.join(pdb_path,pdb_file)

#sasa table
sasa_result = getSASA_final(pdbPATH)
#dssp table
final_dssp_table = getDSSP_final(pdbPATH)
try:
    #pka table
    pka_final_result = getcharge_pKa(pdbPATH)
except:
    problem_pdb.append((pdbname,'getcharge_pKa broke'))
    #load matlab data with dataframe 

f = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/feature_matrix_new/{0}_{1}_individual_feature.mat'.format(pdbname,chain),'r') 
#f2 = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/pdb_rmsd/{0}_individual_feature.mat'.format(pdbname),'r')

datas = f['individual_feature'].value.T
#datas2= f2['individual_feature'].value.T

matlab_data = pd.DataFrame(datas,index=datas[:,0],columns=['resID','eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2',
                                          'Rank_2','GNM_co','co_rank','entropy','rmsf_overall','pca_feature',
                                          'eig_first','eig_sec','shannon_entropy','shannon_entropy20',
                                          'eig5_eig1','atomic_1','atomic_3','atomic_5','ssbond_matrix',
                                          'Polarity','charge','stand_Dcom','side_chain_length',
                                          'ca_len','hole_ss_bond','contact_per_res','gyradius',
                                          'phobic_percent','pca_feature_2','h_bond_group','intrinsicDisorder'])
#matlab_data = matlab_data.assign(RMSD = datas2)

#change resID value type to int
matlab_data['resID'] = matlab_data['resID'].astype('int')

sasa_result.columns = ['resID','sasa']

sasa_result['resID'] = sasa_result['resID'].astype('int')

pka_final_result.columns = ['resID','pka_charge']
pka_final_result['resID'] = pka_final_result['resID'].astype('int')
final_dssp_table.columns = ['resID','dssp_result','loop_percent','sheet_percent','helix_percent']
final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)
final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
# try:
#     final_dssp_table['resID'] = sasa_result['resID'].values
# except:
#     problem_pdb.append((pdbname,'get_dssp_merge broke'))
#     continue    
final_dssp_table['resID'] = final_dssp_table['resID'].astype('int')

#merge matlab feature and python feature
merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')

merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')

merged_data['pka_charge']=merged_data['pka_charge'].fillna(0)

merged_data = merged_data.merge(final_dssp_table,how = 'left',on = 'resID')
#add pdb id
merged_data['PDB_ID'] = pdbname
#change feature order
change_order = ['PDB_ID','resID','atomic_1','atomic_3','atomic_5','ssbond_matrix','Polarity',
                'eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2','Rank_2',
                'GNM_co','co_rank','charge','stand_Dcom','side_chain_length','pka_charge',
                'dssp_result','sasa','entropy','ca_len','hole_ss_bond','rmsf_overall',
                'contact_per_res','gyradius','pca_feature','eig_first','eig_sec',
                'shannon_entropy','shannon_entropy20','eig5_eig1','phobic_percent',
                'loop_percent','h_bond_group','intrinsicDisorder','pca_feature_2','pca_feature_3','sheet_percent','helix_percent']
                                        
pca_3 = merged_data.pca_feature.values/merged_data.pca_feature_2.values

pca_3 = pd.DataFrame(pca_3,columns=['pca_feature_3'])

merged_data = merged_data.join(pca_3)
merged_data = merged_data[change_order]

merged_data.to_csv('/home/tongwade780/pdb_website/python_code/one_pdb_feature/{0}_{1}_all_feature.csv'.format(pdbname,chain),index = False)


propka3.1                                                                                    2018-01-18
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M